In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI"

# Load data into Milvus


In [15]:
# 📌 Cell 1: Import thư viện cần thiết
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from glob import glob
import os

milvus_client = MilvusClient(uri="http://localhost:19530")
collection_name = "my_rag_collection"

# 📌 Cell 2: Load SentenceTransformer model load một lần duy nhất
# Model sẽ được tải và lưu cache tại D:\Big_project_2025
model = SentenceTransformer(
    'all-MiniLM-L6-v2',
    cache_folder="D:/Big_project_2025"
)

# Hàm tiện ích để sinh embeddings cho text
def emb_text(text): 
    return model.encode(text).tolist()


In [16]:
# 📌 Cell 3: Đọc dữ liệu từ file .md
# Lấy toàn bộ file trong thư mục data
text_lines = []
for file_path in glob(r"D:\Big_project_2025\RAG_Milvus\data\*.md", recursive=True):
    with open(file_path, "r", encoding="utf-8") as file:
        file_text = file.read()
    # Cắt text theo header Markdown "# ..."
    text_lines += file_text.split("# ")

# Loại bỏ dòng rỗng + strip khoảng trắng
text_lines = [line.strip() for line in text_lines if line.strip() and line]
print(f"✅ Số đoạn văn lấy được: {len(text_lines)}")


✅ Số đoạn văn lấy được: 8


In [17]:
# 📌 Cell 4: Tạo embeddings cho dữ liệu
embeddings = [emb_text(line) for line in tqdm(text_lines, desc="Creating embeddings")]
print(f"✅ Đã tạo {len(embeddings)} embeddings")

Creating embeddings: 100%|██████████| 8/8 [00:00<00:00, 18.57it/s]

✅ Đã tạo 8 embeddings


In [ ]:
# 📌 Cell 5: Kết nối Milvus
milvus_client = MilvusClient(uri="http://localhost:19530")
collection_name = "my_rag_collection"

# Nếu collection đã tồn tại thì xóa để tạo mới
# if milvus_client.has_collection(collection_name):
#      milvus_client.drop_collection(collection_name)
#      print("⚠️ Đã xóa collection cũ:", collection_name)

⚠️ Đã xóa collection cũ: my_rag_collection


In [19]:
# 📌 Cell 6: Tạo collection mới trong Milvus
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=384,  # 384 = embedding size của all-MiniLM-L6-v2
    metric_type="IP",  # Inner Product (phù hợp cho similarity search)
    consistency_level="Bounded",
)
print("✅ Đã tạo collection:", collection_name)


✅ Đã tạo collection: my_rag_collection


In [20]:
# 📌 Cell 7: Chuẩn bị dữ liệu và insert vào Milvus
data = [
    {"id": i, "vector": vec, "text": text}
    for i, (text, vec) in enumerate(zip(text_lines, embeddings))
]

milvus_client.insert(collection_name=collection_name, data=data)
print(f"🎉 Đã chèn {len(data)} records vào Milvus collection '{collection_name}'")


🎉 Đã chèn 8 records vào Milvus collection 'my_rag_collection'


# triển khai RAG

In [21]:
# 📌 Câu hỏi
question = "câu lạc bộ"

# 📌 Search trong Milvus (chỉ lấy top 1)
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[emb_text(question)],
    limit=1,  # 🔹 Chỉ lấy 1 kết quả tốt nhất
    search_params={"metric_type": "IP", "params": {}},
    output_fields=["text"],
)

# 📌 Lấy text từ kết quả
best_text = search_res[0][0]["entity"]["text"]
#best_distance = search_res[0][0]["distance"]

print("kết quả của:", best_text)
#print("📏 Độ tương đồng:", best_distance)


kết quả của: Câu lạc bộ & Hoạt động ngoại khóa

- CLB Tin học  
- CLB Ngoại ngữ  
- CLB Âm nhạc  
- CLB Thể thao (bóng đá, bóng chuyền, cầu lông)  
- Hoạt động thiện nguyện & phong trào thanh niên tình nguyện.
